# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 
    
for f in file_path_list:
 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
                
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

In [9]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}"""
    )
except Exception as e:
    print(e)

In [10]:
try:
    session.set_keyspace("udacity")
except Exception as e:
    print(e)

In [11]:
query1 = "CREATE TABLE IF NOT EXISTS artist_song_title "
query1 = query1 + "(sessionId int, itemInSession int,artist text, song text, length float,  PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query1)
except Exception as e:
    print(e)
#In this query,the primary key is comprised of two fields:sessionId & itemInSession(partition key & clustering key,respectively).
#The sessionId field partition the columns and within the partition,itemInSession orders the rows within the partition. 

                    

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        query = "INSERT INTO artist_song_title(sessionId, itemInSession,artist, song, length)"
        query = query + "VALUES (%s,%s,%s,%s,%s)"
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

In [15]:
query = "SELECT * FROM artist_song_title WHERE sessionId=338 AND itemInSession=4"
try:
    rows = session.execute(query)
#     session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.song, row.length, row.sessionid, row.iteminsession)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875 338 4


In [16]:
query2 = "CREATE TABLE IF NOT EXISTS song_users "
query2 = query2 + "(userId int, sessionId int, itemInSession int,artist text, song text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query2)
except Exception as e:
    print(e)
#In this query,userId is the partition key with sessionId and itemInSession as the clustering keys.
#userId partitions the columns,while within that partition,sessionId orders the rows with itemInSession sorting the rows.  
                    

In [18]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO song_users (userId, sessionId,itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), line[0], line[9], line[1], line[4], int(line[3])))                    

In [19]:
query = "SELECT * FROM song_users WHERE userId = 10 AND sessionId = 182"

try:
    rows = session.execute(query)
#     session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.song, row.firstname, row.lastname, row.iteminsession)


Down To The Bone Keep On Keepin' On Sylvie Cruz 0
Three Drives Greece 2000 Sylvie Cruz 1
Sebastien Tellier Kilometer Sylvie Cruz 2
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz 3


In [20]:
query3 = "CREATE TABLE IF NOT EXISTS user_name "
query3 = query3 + "(song text,userId int,firstName text, lastName text, PRIMARY KEY (song,userId))"
try:
    session.execute(query3)
except Exception as e:
    print(e)
#This query is set up with the Primary Key of song(partition key) & userId(clustering key).
#Songs partition the columns and the rows are ordered by the userId.    
    

In [21]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
       
        query = "INSERT INTO user_name (song,userId,firstName, lastName)"
        query = query + " VALUES (%s, %s, %s,%s)"
        session.execute(query, (line[1], line[4], line[9]))

In [22]:
query = "SELECT firstName, lastName FROM user_name WHERE song = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
#     session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row)
#     print(row.firstname, row.lastname, row.song)

Row(firstname='Jacqueline', lastname='Lynch')


In [23]:
query = "drop table artist_song_title"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [24]:
session.shutdown()
cluster.shutdown()